Getting the necessary imports

In [74]:
import numpy as np
import pandas as pd
from ast import literal_eval
import os

Reading in all the files to gather distances from

In [92]:
files = os.listdir('../../sorted_csv/')
files.remove('.DS_Store')

This is a cell where we go through the while process of gathering speeds, and turning them into speed csv's. This is all in one big notebook, so this will be commented inside the code.

In [ ]:
for file in files:
    print(file)
    df = pd.read_csv('../../sorted_csv/' + file)

    # This cell changes all string tuples to actual tuples. This is necessary for the distance operation used later
    try:
        df.fillna('(99999, 99999)', inplace = True)
    except:
        pass

    # Creates a new dataframe, converts all tuple strings to integers, and saves the resulting dataframe to df
    new_df = []
    for _, row in df.iterrows():
        sorted_x_coord = sorted([literal_eval(tup) for tup in list(row)])
        new_df.append(sorted_x_coord)
    df = pd.DataFrame.from_records(new_df)
    # This cell turns all (99999, 99999) valued cells back to NAN's so they dont mess up the distance calculations
    df = df.apply(lambda x: x.apply(lambda s: s if s != (99999, 99999) else np.nan))

    # Creating new dataframe to hold the distances
    distances_df = pd.DataFrame(columns = range(df.shape[1]))

    # Calculating the distances between sucessive rows.
    def get_distances(x):
        row1 = df.loc[x]
        row2 = df.loc[x+1]
        dist = []
        # This loop finds the distance by comparing the same column in successive rows.
        for i in range(len(row1)):
            try:
                dist.append(np.sqrt((row1[i][0] - row2[i][0])**2 + (row1[i][1] - row2[i][1])**2))
            except:
                dist.append(99999)
        return dist

    # Looping through the dataframe and assigning all distances to the new dataframe
    for i in range(df.shape[0] - 1):
        distances_df.loc[i] = get_distances(i)

    distances_df.to_csv('../../frame_distances/' + file, index = False)    
    # This counts the values of distances traveled in 1 frame, and counts them all to get data worm by worm.
    counts = []
    for i in range(10):
        counts.append(distances_df.mask((distances_df < i) | (distances_df >= i+1)).count().values)

    # Labels the columns so we dont have an x and y of just numbers
    worm_names = []
    for i in range(len(df.columns)):
        worm_names.append('worm_'+str(i))

    # Finally making and saving the dataframe
    worm_dist = pd.DataFrame(counts, columns = worm_names)

    worm_dist.to_csv('../../total_distance_values/' + file.strip('.csv') + '_distance.csv', index = False)